In [1]:
import pandas as pd
import numpy as np

In [1]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [42]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Вот датасет. Задача - по этим данным о нефтяных месторождениях предсказать, является ли месторождение прибрежным (onshore) или глубоководным (offshore)

In [275]:
df.head()

,Tectonic regime,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,OFFSHORE,OIL,DEVELOPING,INVERSION/WRENCH,3520,NEOGENE,SANDSTONE,2460.0,220.0,20.0,45.0
1,GRAVITY/EXTENSION/EVAPORITE,OFFSHORE,OIL,MATURE PRODUCTION,SALT/PASSIVE MARGIN,9967,CRETACEOUS,LIMESTONE,427.0,160.0,19.0,175.0
2,GRAVITY/EXTENSION/EVAPORITE,ONSHORE,OIL,MATURE PRODUCTION,PASSIVE MARGIN,8700,CRETACEOUS,LIMESTONE,95.0,15.0,12.0,20.0
3,COMPRESSION,ONSHORE,OIL,DECLINING PRODUCTION,THRUST,5084,CRETACEOUS,SANDSTONE,328.0,300.0,13.0,600.0
4,INVERSION/COMPRESSION/EXTENSION,ONSHORE,OIL,DECLINING PRODUCTION,INVERSION/RIFT,1030,CRETACEOUS,SANDSTONE,260.0,33.0,24.0,182.0


Насколько я, не геолог, смог разобраться, здесь есть:
Tectonic regime - тектонический режим, то есть тип тектонических движений в данной местности (что-то вроде сейсмической активноости, видимо). <br>
Hydrocarbon type - тип углеводорода. Тут все ясно, это нефть или газ, еще здесь встречается двуокись углерода и битум. <br>
Reservoir status - статус месторождения. Т.е. разрабатывается он, законсервирован и т.д. <br>
Structural setting - перевод: "структурная установка". То есть это наподобие физической структуры месторождения. <br>
Depth - понятно, глубина залегания.<br>
Period - это геологические периоды, когда то или иное месторождение появилось. Как известно, все углеподороды - это переваренная Землей древняя древесина, но за разное время она переварилась по-разному. <br>
Lithology - тип камней (минералов? не знаю, как сказать грамотнее, но по сути речь идет о камнях) в данной местоности. <br>
Gross - вероятно, объем добычи. <br>
Porosity - пористость (породы). Важный признак, ведь, как я прочитал, породы у нефтяных и газовых месторождений имеют разные значения пористости. <br>
Permeability - проницаемость (породы). С точки зрения физики, это способность пород пласта пропускать сквозь себя жидкость и газы при наличии перепада давления. 

In [5]:
import re

ws = []
for s in test['Period']:
    for w in s.split('/'):
        for ww in w.split('-'):
            ws.append(ww)
        
periods = np.unique(np.array(ws))

for s in df['Period']:
    for w in s.split('/'):
        for ww in w.split('-'):
            ws.append(ww)
        
periods = np.unique(np.array(ws))

Для начала открыл картинку из учебника биологии "Эволюция жизни" и закодировал геологические периоды по их порядковому номеру. Только в датасете встречались не только периоды, но и эры (а эра состоит из нескольких периодов), видимо, если невозможно определить период точнее. Поэтому эры получили некоторый вещественный числовой отрезок, в котором я давал периодом значения исходя из порядкового номера и продолжительности, а если значением в датасете был не период, а эра, то она полачала среднее значение из своего диапазона.

In [6]:
period_encoded = [ \
                  0, 11.7, 17.5, 29, 16, 26,\
                 25, 36, 12.7, 33, 15, 19, 1,\
                 13.6, 23]

Разобраться в схожести остальных фичей было сложнее, да и время было ограничено. Поэтому для них применил обычный Label Encoder и был доволен результатом.

In [9]:
le_rs = LabelEncoder()
le_rs.fit(np.unique(sorted(df['Reservoir status'].unique()) + sorted(test['Reservoir status'].unique())))

LabelEncoder()

In [10]:
ws = []
for s in np.unique( sorted(df['Tectonic regime'].unique()) + sorted(test['Tectonic regime'].unique()) ):
    for w in s.split('/'):
        ws.append(w)
        
le_tr = LabelEncoder()
le_tr.fit(np.unique(ws))

LabelEncoder()

In [11]:
ws = []
for s in np.unique( sorted(df['Structural setting'].unique()) + sorted(test['Structural setting'].unique()) ):
    for w in s.split('/'):
        ws.append(w)
        
le_ss = LabelEncoder()
le_ss.fit(np.unique(ws))

LabelEncoder()

In [279]:
sorted(df['Hydrocarbon type'].unique()) + sorted(test['Hydrocarbon type'].unique())

['CARBON DIOXIDE',
 'GAS',
 'GAS-CONDENSATE',
 'METHANE HYDRATE',
 'OIL',
 'BITUMEN',
 'CARBON DIOXIDE',
 'GAS',
 'GAS-CONDENSATE',
 'OIL']

In [12]:
le_ht = LabelEncoder()
le_ht.fit(np.unique( sorted(df['Hydrocarbon type'].unique()) + sorted(test['Hydrocarbon type'].unique())))

LabelEncoder()

In [13]:
le_l = LabelEncoder()
le_l.fit(np.unique( sorted(df['Lithology'].unique()) + sorted(test['Lithology'].unique())))

LabelEncoder()

In [54]:
#df = pd.read_csv('train.csv')
for i in range(len(df)):
    print(df['Period'][i])
    for w in df['Period'][i].split('/'):
        for ww in w.split('-'):
            df['Period'][i] = (period_encoded[np.where(periods == ww)[0][0]])


NEOGENE
CRETACEOUS
CRETACEOUS
CRETACEOUS
CRETACEOUS
CARBONIFEROUS
PROTEROZOIC
CRETACEOUS
CARBONIFEROUS
PERMIAN
CRETACEOUS
PALEOGENE-NEOGENE
DEVONIAN
NEOGENE
CRETACEOUS
JURASSIC
NEOGENE
CRETACEOUS
CARBONIFEROUS
CARBONIFEROUS
JURASSIC
PERMIAN
NEOGENE
JURASSIC
JURASSIC
PERMIAN
CARBONIFEROUS
PALEOGENE
JURASSIC
CRETACEOUS
CARBONIFEROUS-PERMIAN
CRETACEOUS-PALEOGENE
NEOGENE
DEVONIAN
DEVONIAN
JURASSIC
JURASSIC
CRETACEOUS
NEOGENE
DEVONIAN
PERMIAN
CRETACEOUS
CRETACEOUS
JURASSIC
ORDOVICIAN
CRETACEOUS
NEOGENE
DEVONIAN
DEVONIAN
NEOGENE
CRETACEOUS
CARBONIFEROUS
TRIASSIC
JURASSIC
CRETACEOUS
PALEOGENE
JURASSIC
JURASSIC
CRETACEOUS
PERMIAN
CRETACEOUS
PROTEROZOIC-CAMBRIAN
PALEOGENE
JURASSIC
CARBONIFEROUS
CRETACEOUS
PALEOGENE
PALEOGENE
CRETACEOUS
JURASSIC
NEOGENE
PALEOGENE
PERMIAN
TRIASSIC-JURASSIC
CRETACEOUS
JURASSIC
CRETACEOUS
CARBONIFEROUS-PERMIAN
NEOGENE
JURASSIC
CRETACEOUS
CRETACEOUS
JURASSIC
JURASSIC-CRETACEOUS
TRIASSIC
DEVONIAN
DEVONIAN
DEVONIAN
CRETACEOUS
CRETACEOUS
JURASSIC
CRETACEOUS
CARBONIFERO

/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [325]:
df['Reservoir status'].unique()

array(['DEVELOPING', 'MATURE PRODUCTION', 'DECLINING PRODUCTION',
       'REJUVENATING', 'ABANDONED', 'NEARLY DEPLETED', 'UNKNOWN',
       'PLATEAU PRODUCTION', 'UNDEVELOPED', 'CONTINUING DEVELOPMENT',
       'SECOND PLATEAU PRODUTION', 'DEPLETED'], dtype=object)

В данных было 2 подвоха. <br>
1. В тестовых данных присустсоввали значения, которых не было в обучающих, поэтому энкодеры надо было обучать на совокупности значений из обучения и теста. <br>
2. В нескольких столбцах было много значений, в которых было не одно значения, а перечислялось несколько через слэш. Например, иногда в типе углеводородов была не просто "нефть" или "газ, а "нефть/газ", то есть, видимо, и то и другое. Эффективным оказалось в таких случаях суммировать закодированные значения. 

In [43]:
for i in range(len(df)):
    new = []
    for w in df['Tectonic regime'][i].split('/'):
        new.append(le_tr.transform([w]))
    df['Tectonic regime'][i] = np.sum(new)
    new = []
    for w in df['Structural setting'][i].split('/'):
        new.append(le_ss.transform([w]))
    df['Structural setting'][i] = np.sum(new)
    new = []
    for w in df['Hydrocarbon type'][i].split('/'):
        new.append(le_ht.transform([w]))
    df['Hydrocarbon type'][i] = np.sum(new)
    new = []
    for w in df['Lithology'][i].split('/'):
        new.append(le_l.transform([w]))
    df['Lithology'][i] = np.sum(new)
    new = []
    for w in df['Reservoir status'][i].split('/'):
        new.append(le_rs.transform([w]))
    df['Reservoir status'][i] = np.sum(new)
    

/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/leonidts/ana

In [55]:
X = df.drop(['Onshore/Offshore'], axis=1)

In [334]:
type(X['Tectonic regime'][0])

str

In [47]:
scaler.fit(X[['Depth', 'Gross', 'Netpay', 'Porosity', 'Permeability']])
X[['Depth', 'Gross', 'Netpay', 'Porosity', 'Permeability']] = scaler.transform(X[['Depth', 'Gross', 'Netpay', 'Porosity', 'Permeability']])

На данный момент я был не уверен, какой алгоритм сработает лучше, поэтому нормализовал числовые данные.

In [48]:
target = df['Onshore/Offshore']

In [49]:
target = LabelEncoder().fit_transform(target)

In [50]:
target

array([0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       2, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,

In [56]:
for c in X.columns:
    X[c] = X[c].astype(float)

Самые высокие результаты показал классификатор XGBoost. Метрикой в соревновании была accuracy, поэтому опирался я именно на нее.

In [57]:
import xgboost as xgb

XGBR = xgb.XGBClassifier(max_depth=6, objective='multi:softmax', n_estimators=1000, num_class = 3)

from sklearn.model_selection import cross_val_predict, StratifiedKFold
kf = StratifiedKFold(n_splits = 5, shuffle = True)
#predicted = cross_val_predict(XGBR, features, labels_16, cv = kf, n_jobs = 2)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.1)
XGBR.fit(X_train, y_train)
predicted = XGBR.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
print(accuracy_score(y_test, predicted))
#print(cross_val_score(XGBR, X_test, y_test, predicted))

0.8709677419354839


Далее идет обработка теста и предсказание, так что итог подведу здесь. <br>
Решение оказалось отличным, поскольку я получил 43/50 очков (86%).

In [58]:
predicted

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1])

In [81]:
import xgboost as xgb

XGBR = xgb.XGBRegressor(max_depth=6, objective='multi:softmax', n_estimators=1000, num_class = 3)

from sklearn.model_selection import cross_val_predict, StratifiedKFold
kf = StratifiedKFold(n_splits = 5, shuffle = True)
#predicted = cross_val_predict(XGBR, features, labels_16, cv = kf, n_jobs = 2)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.1)
XGBR.fit(X_train, y_train)
predicted = XGBR.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
print(accuracy_score(y_test, predicted))
#print(cross_val_score(XGBR, X_test, y_test, predicted))

0.9032258064516129


In [59]:
test = pd.read_csv('test.csv')

In [60]:
for i in range(len(test)):
    print(test['Period'][i])
    for w in test['Period'][i].split('/'):
        for ww in w.split('-'):
            test['Period'][i] = (period_encoded[np.where(periods == ww)[0][0]])

PERMIAN
PERMIAN
JURASSIC
NEOGENE
CARBONIFEROUS
CRETACEOUS
CARBONIFEROUS
CARBONIFEROUS
CRETACEOUS
NEOGENE
JURASSIC
CRETACEOUS
PALEOGENE
TRIASSIC
JURASSIC
PERMIAN
JURASSIC
PALEOGENE
JURASSIC
CRETACEOUS
JURASSIC
NEOGENE
CRETACEOUS
NEOGENE
DEVONIAN-CARBONIFEROUS
CARBONIFEROUS
CARBONIFEROUS
CRETACEOUS
DEVONIAN
JURASSIC
NEOGENE
PALEOZOIC-CRETACEOUS
CARBONIFEROUS-PERMIAN
NEOGENE
SILURIAN
NEOGENE
PALEOGENE
CRETACEOUS
CARBONIFEROUS-PERMIAN
PERMIAN
NEOGENE
CRETACEOUS
CRETACEOUS
CARBONIFEROUS
CRETACEOUS
PERMIAN
JURASSIC
PERMIAN
CARBONIFEROUS
JURASSIC
NEOGENE
JURASSIC
CRETACEOUS
NEOGENE
CRETACEOUS
PERMIAN
SILURIAN
PERMIAN
CAMBRIAN-ORDOVICIAN
SILURIAN
DEVONIAN
ORDOVICIAN
CRETACEOUS
DEVONIAN
JURASSIC
NEOGENE
PALEOGENE
NEOGENE
DEVONIAN
PERMIAN
CARBONIFEROUS
TRIASSIC
CRETACEOUS
CRETACEOUS
PERMIAN
PROTEROZOIC
PALEOGENE
CRETACEOUS
NEOGENE
CRETACEOUS
CRETACEOUS
NEOGENE
PALEOGENE
CARBONIFEROUS
CRETACEOUS
CARBONIFEROUS
CRETACEOUS
JURASSIC
CRETACEOUS
PERMIAN
CRETACEOUS
JURASSIC
PERMIAN
CRETACEOUS
CARBONIFER

/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [61]:
for i in range(len(test)):
    new = []
    for w in test['Tectonic regime'][i].split('/'):
        new.append(le_tr.transform([w]))
    test['Tectonic regime'][i] = np.sum(new)
    new = []
    for w in test['Structural setting'][i].split('/'):
        new.append(le_ss.transform([w]))
    test['Structural setting'][i] = np.sum(new)
    new = []
    for w in test['Hydrocarbon type'][i].split('/'):
        new.append(le_ht.transform([w]))
    test['Hydrocarbon type'][i] = np.sum(new)
    new = []
    for w in test['Lithology'][i].split('/'):
        new.append(le_l.transform([w]))
    test['Lithology'][i] = np.sum(new)
    new = []
    for w in test['Reservoir status'][i].split('/'):
        new.append(le_rs.transform([w]))
    test['Reservoir status'][i] = np.sum(new)
    

/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/leonidts/ana

In [62]:
for c in test.columns:
    test[c] = test[c].astype(float)

In [102]:
predicted = XGBR.predict(test)

In [103]:
pred = pd.DataFrame(predicted)

In [424]:
pred[0].unique()

array([1, 0])

In [67]:
pred[pred[0] == 0][0] = 'OFFSHORE'
pred[pred[0] == 1][0] = 'ONSHORE'

/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/leonidts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [104]:
pred[0] = pred[0].apply(lambda x: 'OFFSHORE' if x==0 else 'ONSHORE')

In [105]:
pred

,0
0,ONSHORE
1,ONSHORE
2,ONSHORE
3,OFFSHORE
4,ONSHORE
...,...
128,OFFSHORE
129,ONSHORE
130,OFFSHORE
131,ONSHORE


In [106]:
pred.to_csv('prediction.csv', index=False, header=False)